In [ ]:
import sys
relative_root = "../.."
sys.path.append(relative_root)

In [ ]:
import joblib
import rich
import pandas as pd
from sklearn.metrics import accuracy_score
import json
from examples.training import titanic_config
from pathlib import Path

## Load data & model

In [ ]:
try:
    preprocessed_train = pd.read_csv(Path(relative_root) / titanic_config.LOCAL_TRAIN_FILENAME)
    preprocessed_test = pd.read_csv(Path(relative_root) / titanic_config.LOCAL_TEST_FILENAME)
    rf_model = joblib.load(Path(relative_root) / titanic_config.LOCAL_MODEL_FILENAME)
    with open(Path(relative_root) / titanic_config.LOCAL_FI_FILENAME, "r") as file:
        feature_importance = json.loads(file.read())
except FileNotFoundError:
    raise FileNotFoundError("To generate these files, run `make train-titanic`")

In [ ]:
rf_model.predict(preprocessed_test)[0]  # test model inference

## Init trubrics context

In [ ]:
from trubrics.context import DataContext

In [ ]:
data_context = DataContext(
    name="my_first_dataset",
    version=0.1,
    testing_data=preprocessed_test,
    target=titanic_config.TARGET,
    training_data=preprocessed_train,
    minimum_functionality_data=preprocessed_test.head(),
)

## Init trubrics validator

In [ ]:
from trubrics.validations import ModelValidator
from examples.cli.custom_scorer import custom_scorers  # see here for custom_scorer example

In [ ]:
model_validator = ModelValidator(data=data_context, model=rf_model, custom_scorers=custom_scorers)

## Use the trubrics validator to create out-of-the-box validations

In [ ]:
minimum_functionality = [
    model_validator.validate_minimum_functionality(severity="warning")  # validation severity can be passed in to any validation
]
_ = [rich.print(val.dict()) for val in minimum_functionality]

In [ ]:
import sklearn.metrics
rich.print("View sklearn default scorers: ", sklearn.metrics.SCORERS)

In [ ]:
performance = [
    model_validator.validate_performance_against_threshold(metric="accuracy", threshold=0.8),
    model_validator.validate_performance_against_threshold(metric="recall", threshold=0.7, severity="warning"),
    model_validator.validate_performance_against_threshold(metric="precision", threshold=0.7, severity="error"),
    model_validator.validate_performance_against_threshold(metric="f1", threshold=0.7, severity="experiment"),
    model_validator.validate_performance_against_threshold(metric="my_custom_loss", threshold=-0.7, severity="experiment"),
    model_validator.validate_performance_against_dummy(metric="f1"),
    model_validator.validate_performance_against_dummy(metric="recall", strategy="stratified"),
]
_ = [rich.print(val.dict()) for val in performance]

In [ ]:
fairness = [
    model_validator.validate_biased_performance_across_category(metric="accuracy", category="Embarked", threshold=0.1),
    model_validator.validate_biased_performance_across_category(metric="precision", category="Sex", threshold=0.05)
]
_ = [rich.print(val.dict()) for val in fairness]

In [ ]:
explainability = [
    model_validator.validate_feature_in_top_n_important_features(feature="Sex_female", feature_importance=feature_importance, top_n_features=3),
    model_validator.validate_feature_in_top_n_important_features(feature="Age", feature_importance=feature_importance, top_n_features=2)
]
_ = [rich.print(val.dict()) for val in explainability]

## Collect user feedback from app

In [ ]:
from trubrics.utils.loader import get_business_feedback_data
# read test data and run single outlier test
try:
    data = get_business_feedback_data(tracking=False)
    display(data)
except FileNotFoundError:
    print("Please generate feedback from the streamlit app in order to read it back here")

--> **DS response: "It isn't normal, the model should not be more accurate for different groups of people. I'll add a test for this."**

## Create custom validations

In [ ]:
from examples.cli.custom_validator import CustomValidator  # see script for CustomValidator example

In [ ]:
model_custom_validator = CustomValidator(data=data_context, model=rf_model, custom_scorers=custom_scorers)

In [ ]:
custom = [model_custom_validator.validate_performance_for_different_fares(fare_cutoff=25, severity="warning")]
_ = [rich.print(val.dict()) for val in custom]

## Save trubric

In [ ]:
from trubrics.context import TrubricContext

validations = minimum_functionality + performance + fairness + explainability + custom

trubric_context = TrubricContext(
    trubric_name="my_first_trubric",
    model_name="my_model",
    data_context_name=data_context.name,
    data_context_version=data_context.version,
    metadata={"tag": "master"},
    validations=validations,
)

In [ ]:
# save trubric to a local .json
trubric_context.save_local(path="../data")

In [ ]:
# or save to trubrics UI
user_id = None  # enter User ID from trubrics manager here
url = None  # enter api url for trubrics manager here

try:
    trubric_context.save_ui(url=url, user_id=user_id)
    print("Trubric saved to UI.")
except Exception as e:
    print(f"Error in saving to trubrics manager:\n{e}")

## Execute trubric from file

In [ ]:
from trubrics.validations.run import run_trubric

In [ ]:
trubric = TrubricContext.parse_file("../data/my_first_trubric.json")

all_validation_results = run_trubric(
    data_context=data_context,
    model=rf_model,
    custom_validator=CustomValidator,
    custom_scorers=custom_scorers,
    trubric=trubric
)

In [ ]:
for validation_result in all_validation_results:
    rich.print(f"{validation_result.validation_type} - {validation_result.severity.upper()}.......{validation_result.outcome}")

In [ ]:
# or run from cli
import os
os.chdir("../..")
!make example-run-trubric
os.chdir("examples/notebooks")